In [101]:
# from google.colab import drive
# drive.mount('/content/drive')

In [127]:
import pandas as pd

ratings = pd.read_csv('/Users/snapp/PycharmProjects/IMDb-IR-System/Logic/core/recommender_system/data/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/Users/snapp/PycharmProjects/IMDb-IR-System/Logic/core/recommender_system/data/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [128]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [129]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [130]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [131]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [132]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
title_to_index = pd.Series(movies.index, index=movies['title']).drop_duplicates()


In [133]:
def genre_recommendations(title, n=10):
    idx = title_to_index[title]
    
    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the n most similar movies
    sim_scores = sim_scores[1:n+1]
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Return the top n most similar movies
    return movies['title'].iloc[movie_indices]

In [134]:
# Example
genre_recommendations('GoldenEye (1995)')

345     Clear and Present Danger (1994)
543           Surviving the Game (1994)
724                    Rock, The (1996)
788                     Daylight (1996)
825               Chain Reaction (1996)
978                 Maximum Risk (1996)
1467                    Anaconda (1997)
1513                     Con Air (1997)
1693                   Firestorm (1998)
3686          Perfect Storm, The (2000)
Name: title, dtype: object


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [135]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [136]:
data = pd.merge(ratings, movies, on='movie_id')

In [137]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,1,661,3,0,660,James and the Giant Peach (1996),"['Animation', ""Children's"", 'Musical']"
2,1,914,3,0,913,My Fair Lady (1964),"['Musical', 'Romance']"
3,1,3408,4,0,3407,Erin Brockovich (2000),['Drama']
4,1,2355,5,0,2354,"Bug's Life, A (1998)","['Animation', ""Children's"", 'Comedy']"


In [138]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

# Create User-Genre Matrix
user_genre_matrix = data.pivot_table(index='user_id', columns='genres', values='rating', aggfunc='mean').fillna(0)
user_genre_matrix = user_genre_matrix.div(user_genre_matrix.sum(axis=1), axis=0)

# Calculate User Similarity
user_similarity = cosine_similarity(user_genre_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_genre_matrix.index, columns=user_genre_matrix.index)


In [139]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,1,661,3,0,660,James and the Giant Peach (1996),"['Animation', ""Children's"", 'Musical']"
2,1,914,3,0,913,My Fair Lady (1964),"['Musical', 'Romance']"
3,1,3408,4,0,3407,Erin Brockovich (2000),['Drama']
4,1,2355,5,0,2354,"Bug's Life, A (1998)","['Animation', ""Children's"", 'Comedy']"


In [140]:
def get_top_n_similar_users(user_id, n=5):
    if user_id not in user_similarity_df.index:
        raise ValueError("User ID not found in the user similarity matrix.")
    
    similarity_scores = user_similarity_df[user_id]
    
    similar_users = similarity_scores.sort_values(ascending=False).drop(user_id)
    
    return similar_users.head(n).index.tolist()

In [141]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    similar_users = get_top_n_similar_users(user_id, n_similar_users)
    
    similar_users_ratings = data[data['user_id'].isin(similar_users)]
    
    target_user_ratings = data[data['user_id'] == user_id]
    watched_movies = target_user_ratings['movie_id'].tolist()
    
    recommendations = similar_users_ratings[~similar_users_ratings['movie_id'].isin(watched_movies)]
    
    top_recommendations = recommendations.groupby('movie_id').agg({'rating': 'mean'}).sort_values('rating', ascending=False).head(n_recommendations).index
    
    return movies[movies['movie_id'].isin(top_recommendations)][['title', 'genres']]

In [142]:
# Example
user_id = 2
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [143]:
recommended_movies

,title,genres
821,Ransom (1996),"['Drama', 'Thriller']"
891,Vertigo (1958),"['Mystery', 'Thriller']"
896,North by Northwest (1959),"['Drama', 'Thriller']"
1891,"Last Emperor, The (1987)","['Drama', 'War']"
2255,Life Is Beautiful (La Vita è bella) (1997),"['Comedy', 'Drama']"
2260,American History X (1998),['Drama']
2267,Elizabeth (1998),['Drama']
2283,"Big Chill, The (1983)","['Comedy', 'Drama']"
2315,Babe: Pig in the City (1998),"[""Children's"", 'Comedy']"
2473,"Lock, Stock & Two Smoking Barrels (1998)","['Comedy', 'Crime', 'Thriller']"


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [144]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [145]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
# calculate rating matrix

In [146]:
from scipy.sparse.linalg import svds

Ratings = Ratings.values.astype(float)

U, sigma, Vt = svds(Ratings, k=50)
sigma = np.diag(sigma)

In [147]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [148]:
preds = pd.DataFrame(all_user_predicted_ratings, columns=ratings.pivot(index='user_id', columns='movie_id', values='rating').columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.292556,0.164772,-0.184504,-0.018528,0.021516,-0.181793,-0.102936,0.157111,-0.058523,-0.164231,...,0.032475,0.007048,0.033170,-0.007577,-0.075329,0.394578,0.124714,0.051545,0.057349,0.076743
1,0.754817,0.128981,0.341128,0.009540,0.001830,1.314580,0.077427,0.062218,0.163983,1.514572,...,-0.051376,-0.015780,-0.010918,0.055628,-0.016037,0.166941,-0.421144,-0.106675,-0.049650,-0.126564
2,1.844858,0.473855,0.098573,-0.039309,-0.019895,-0.154038,-0.141531,0.111551,0.035977,0.738427,...,0.049994,0.003596,0.022734,0.043377,0.034825,0.120205,0.086553,0.034783,0.029337,-0.121871
3,0.395484,-0.045487,0.033716,0.083890,0.051561,0.260676,-0.081541,0.023891,0.051932,-0.077170,...,0.010847,0.007518,0.004269,0.009527,-0.079630,0.077845,0.051369,-0.017046,0.018980,-0.047580
4,1.557609,-0.006700,-0.044860,0.249014,-0.043320,1.518140,-0.164306,-0.043626,-0.078771,0.424064,...,0.097708,0.013847,-0.024256,-0.043126,-0.065997,-0.025571,0.517685,0.008508,0.106630,0.221219


In [149]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):

    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = predictions.iloc[user_row_number].sort_values(ascending=False)


    user_data = original_ratings[original_ratings.user_id == userID]
    
    # merfe movies data with user_data
    user_full = (user_data.merge(movies, how='left', left_on='movie_id', right_on='movie_id')
                         .sort_values(['rating'], ascending=False))


    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending highest {num_recommendations} predicted ratings movies not already rated.')

    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
                       merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                             left_on='movie_id', right_on='movie_id').
                       rename(columns={user_row_number: 'Predictions'}).
                       sort_values('Predictions', ascending=False).
                       iloc[:num_recommendations, :-1])

    return user_full, recommendations

In [150]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [151]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [152]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),['Drama']
285,4375,175,5,4374,174,Kids (1995),['Drama']
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)","['Action', 'Drama', 'Romance', 'Thriller']"
65,4375,3809,5,4374,3808,What About Bob? (1991),['Comedy']
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)","['Drama', 'Thriller']"
270,4375,150,5,4374,149,Apollo 13 (1995),['Drama']
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",['Horror']
125,4375,349,5,4374,348,Clear and Present Danger (1994),"['Action', 'Adventure', 'Thriller']"
60,4375,288,5,4374,287,Natural Born Killers (1994),"['Action', 'Thriller']"
141,4375,47,5,4374,46,Seven (Se7en) (1995),"['Crime', 'Thriller']"


#### Evaluation

In [153]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

train_ratings = train_data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
R_train = train_ratings.values
user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train = svds(Ratings_demeaned_train, k=50)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train) + user_ratings_mean_train.reshape(-1, 1)

preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)

def predict_rating(user_id, movie_id):
    try:
        user_row_number = user_id - 1  # User ID starts at 1, not 0
        pred = preds_train.loc[user_row_number, movie_id]
    except KeyError:
        # If the movie_id is not in the training set, return NaN
        pred = np.nan
    return pred

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 2.7404660761407382


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [154]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [155]:
ratings_df = pd.read_csv('/Users/snapp/PycharmProjects/IMDb-IR-System/Logic/core/recommender_system/data/ratings.csv', sep='\t', encoding='latin-1',
                         usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies_df = pd.read_csv('/Users/snapp/PycharmProjects/IMDb-IR-System/Logic/core/recommender_system/data/movies.csv', sep='\t', encoding='latin-1',
                        usecols=['movie_id', 'title', 'genres'])

# Ensure user_id and movie_id are zero-indexed
ratings_df['user_id'] = ratings_df['user_id'].astype('category').cat.codes.values
ratings_df['movie_id'] = ratings_df['movie_id'].astype('category').cat.codes.values

num_users = ratings_df['user_id'].nunique()
num_movies = ratings_df['movie_id'].nunique()


# Define the dataset
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        user = self.ratings.iloc[idx, 0]
        movie = self.ratings.iloc[idx, 1]
        rating = self.ratings.iloc[idx, 4]
        return torch.tensor(user, dtype=torch.long), torch.tensor(movie, dtype=torch.long), torch.tensor(rating, dtype=torch.float)

class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, user, movie):
        user_emb = self.user_embedding(user)
        movie_emb = self.movie_embedding(movie)
        x = torch.cat([user_emb, movie_emb], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create the dataset and dataloader
dataset = RatingsDataset(ratings_df)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function and optimizer
model = RecommenderNet(num_users, num_movies)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for users, movies, ratings in dataloader:
        optimizer.zero_grad()
        outputs = model(users, movies)
        loss = criterion(outputs.squeeze(), ratings)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')

# Evaluation
model.eval()
user_ids = torch.tensor(ratings_df['user_id'].values, dtype=torch.long)
movie_ids = torch.tensor(ratings_df['movie_id'].values, dtype=torch.long)
ratings_values = torch.tensor(ratings_df['rating'].values, dtype=torch.float)

with torch.no_grad():
    predictions = model(user_ids, movie_ids).squeeze()
    rmse = sqrt(mean_squared_error(ratings_values.numpy(), predictions.numpy()))
    print(f'Root Mean Squared Error: {rmse}')


Epoch 1, Loss: 131785.38130632308
Epoch 2, Loss: 1727.8480827462856
Epoch 3, Loss: 812.0619988592491
Root Mean Squared Error: 2156.629430384367


In [156]:
# Example

sample_user_id = 2
sample_user_data = ratings_df[ratings_df['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

Sample predictions for user ID 2:
Movie ID: 3189, Predicted Rating: 3441.901123046875, Actual Rating: 3420.0
Movie ID: 1504, Predicted Rating: 1659.08154296875, Actual Rating: 1640.0
Movie ID: 627, Predicted Rating: 661.005615234375, Actual Rating: 647.0
Movie ID: 1295, Predicted Rating: 1413.759765625, Actual Rating: 1393.0
Movie ID: 3301, Predicted Rating: 3522.430908203125, Actual Rating: 3533.0
Movie ID: 101, Predicted Rating: 107.56241607666016, Actual Rating: 103.0
Movie ID: 2530, Predicted Rating: 2740.603759765625, Actual Rating: 2734.0
Movie ID: 1120, Predicted Rating: 1211.33251953125, Actual Rating: 1209.0
Movie ID: 1327, Predicted Rating: 1463.689453125, Actual Rating: 1430.0
Movie ID: 3622, Predicted Rating: 3885.011962890625, Actual Rating: 3867.0
Movie ID: 1007, Predicted Rating: 1107.129638671875, Actual Rating: 1078.0
Movie ID: 2785, Predicted Rating: 3053.85595703125, Actual Rating: 2996.0
Movie ID: 1483, Predicted Rating: 1633.41552734375, Actual Rating: 1614.0
Movie

In [160]:
def predict_unrated_movies(user_id, model, ratings, movies):
    all_movie_ids = set(movies['movie_id'].unique())
    
    rated_movie_ids = set(ratings[ratings['user_id'] == user_id]['movie_id'].unique())
    
    unrated_movie_ids = list(all_movie_ids - rated_movie_ids)
    
    max_movie_id = model.movie_embedding.num_embeddings
    valid_unrated_movie_ids = [movie_id for movie_id in unrated_movie_ids if movie_id < max_movie_id]

    user_ids = torch.tensor([user_id] * len(valid_unrated_movie_ids), dtype=torch.long)
    movie_ids = torch.tensor(valid_unrated_movie_ids, dtype=torch.long)
    
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(len(movie_ids)):
            user = user_ids[i].unsqueeze(0)
            movie = movie_ids[i].unsqueeze(0)
            prediction = model(user, movie).squeeze().item()
            predictions.append(prediction)
    
    unrated_movies = movies[movies['movie_id'].isin(valid_unrated_movie_ids)].copy()
    unrated_movies.loc[:, 'predicted_rating'] = predictions
    top_recommendations = unrated_movies.sort_values('predicted_rating', ascending=False)
    
    return top_recommendations

# Example
user_id = 3
predictions = predict_unrated_movies(user_id, model, ratings_df, movies_df)
print(predictions)

      movie_id                                    title  \
3606      3675                   White Christmas (1954)   
3625      3694       Toxic Avenger, Part II, The (1989)   
3621      3690                   Porky's Revenge (1985)   
3584      3653               Endless Summer, The (1966)   
3576      3645  Cleo From 5 to 7 (Cléo de 5 à 7) (1962)   
...        ...                                      ...   
5            6                              Heat (1995)   
3            4                 Waiting to Exhale (1995)   
1            2                           Jumanji (1995)   
2            3                  Grumpier Old Men (1995)   
0            1                         Toy Story (1995)   

                            genres  predicted_rating  
3606                       Musical       4012.373047  
3625                 Comedy|Horror       3982.047119  
3621                        Comedy       3981.729492  
3584                   Documentary       3976.490479  
3576            

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [170]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture

file_path = "/Users/snapp/PycharmProjects/IMDb-IR-System/Logic/core/recommender_system/data/gmm-dataset.csv"
df = pd.read_csv(file_path)


if len(df.columns[1:]) != 10:
    raise ValueError("The dataset must contain exactly 10 distinct countries.")

# Extract the URLs and click data
urls = df.iloc[:, 0]
click_data_matrix = df.iloc[:, 1:]

# Normalize the number of clicks to account for different scales
click_data_matrix = click_data_matrix.apply(lambda x: (x - x.mean()) / x.std(), axis=0)

# Define the number of clusters you want to find (this can be tuned)
n_clusters = 5  # Example: 5 clusters

# Fit the Gaussian Mixture Model (GMM) to the click data matrix
gmm = GaussianMixture(n_components=n_clusters, random_state=42)
gmm.fit(click_data_matrix)

# Analyze the resulting model by calculating the log likelihood, BIC, and AIC metrics
log_likelihood = gmm.score(click_data_matrix)
bic = gmm.bic(click_data_matrix)
aic = gmm.aic(click_data_matrix)

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

Log-Likelihood: -11.301520932782887
BIC: 3775.4051777466593
AIC: 2918.3041865565774
Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x3232ae980> (for post_run_cell), with arguments args (<ExecutionResult object at 318ca70e0, execution_count=170 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 318b79370, raw_cell="import pandas as pd
import numpy as np
from sklear.." store_history=True silent=False shell_futures=True cell_id=None> result=None>,),kwargs {}:


AttributeError: module 'IPython.utils.py3compat' has no attribute 'string_types'